# The Lancet

In [1]:
from selenium import webdriver
import requests
import pandas as pd
from bs4 import BeautifulSoup
import xmltodict
import threading


In [2]:
out = []
seen_papers ={}
lancet_rss_feeds = ['https://www.thelancet.com/rssfeed/ebiom_current.xml','https://thelancet.com/rssfeed/eclinm_current.xml', 'https://www.thelancet.com/rssfeed/lancet_current.xml','https://www.thelancet.com/rssfeed/lanrhe_current.xml','https://www.thelancet.com/rssfeed/lanres_current.xml','https://www.thelancet.com/rssfeed/lanwpc_current.xml',
                    'https://www.thelancet.com/rssfeed/lansea_current.xml','https://www.thelancet.com/rssfeed/lanepe_current.xml','https://www.thelancet.com/rssfeed/lanam_current.xml','https://www.thelancet.com/rssfeed/lanpub_current.xml','https://www.thelancet.com/rssfeed/lanpsy_current.xml','https://www.thelancet.com/rssfeed/lanplh_current.xml',
                    'https://www.thelancet.com/rssfeed/lanonc_current.xml','https://www.thelancet.com/rssfeed/laneur_current.xml','https://www.thelancet.com/rssfeed/lanmic_current.xml','https://www.thelancet.com/rssfeed/laninf_current.xml','https://www.thelancet.com/rssfeed/lanhl_current.xml','https://www.thelancet.com/rssfeed/lanhiv_current.xml',
                    'https://www.thelancet.com/rssfeed/lanhae_current.xml','https://www.thelancet.com/rssfeed/langlo_current.xml','https://www.thelancet.com/rssfeed/langas_current.xml','https://www.thelancet.com/rssfeed/landig_current.xml','https://www.thelancet.com/rssfeed/landia_current.xml','https://www.thelancet.com/rssfeed/lanchi_current.xml',
                    'https://www.thelancet.com/rssfeed/lanrhe_online.xml','https://www.thelancet.com/rssfeed/lanres_online.xml','https://www.thelancet.com/rssfeed/lanwpc_online.xml','https://www.thelancet.com/rssfeed/lansea_online.xml','https://www.thelancet.com/rssfeed/lanepe_online.xml','https://www.thelancet.com/rssfeed/lanam_online.xml',
                    'https://www.thelancet.com/rssfeed/lanpub_online.xml','https://www.thelancet.com/rssfeed/lanpsy_online.xml','https://www.thelancet.com/rssfeed/lanplh_online.xml','https://www.thelancet.com/rssfeed/lanonc_online.xml','https://www.thelancet.com/rssfeed/lanmic_online.xml','https://www.thelancet.com/rssfeed/laneur_online.xml',
                    'https://www.thelancet.com/rssfeed/laninf_online.xml','https://www.thelancet.com/rssfeed/lanhiv_online.xml','https://www.thelancet.com/rssfeed/lanhl_online.xml','https://www.thelancet.com/rssfeed/lanhae_online.xml','https://www.thelancet.com/rssfeed/langlo_online.xml','https://www.thelancet.com/rssfeed/langas_online.xml',
                    'https://www.thelancet.com/rssfeed/landig_online.xml','https://www.thelancet.com/rssfeed/landia_online.xml','https://www.thelancet.com/rssfeed/lanchi_online.xml','https://www.thelancet.com/rssfeed/lancet_online.xml','https://www.thelancet.com/rssfeed/eclinm_online.xml','https://www.thelancet.com/rssfeed/ebiom_online.xml']

In [3]:
def get_lancet_info(paper):
    global out
    global seen_papers
    paper_info = {}
    paper_info['paper_id'] = paper['dc:identifier']
    if paper_info['paper_id'] in seen_papers:
        return
    paper_info['date']= paper['dc:date']
    paper_info['source'] = paper['prism:publicationName']
    paper_info['title'] =paper['dc:title']
    paper_info['abstract'] = paper['description']
    paper_info['url'] = paper['link'].replace('?rss=yes', '')
    driver = webdriver.Chrome()
    driver.get(paper_info['url'])
    html = driver.page_source
    driver.close()

    soup=BeautifulSoup(html,'html.parser')
    authors=[]
    affiliations = []
    paper_info['subjects'] = [x.text for x in soup.find_all('ul', class_='rlist keywords-list inline-bullet-list')]
    for auth in soup.find_all('li', class_='loa__item author'):
        authors.append(auth.find('a', class_='loa__item__name article-header__info__ctrl loa__item__email').text)
        affil_lst = auth.find_all('div', class_='article-header__info__group__body')
        for i in range(len(affil_lst)):
            affiliations.append(affil_lst[i].text.strip())
    paper_info['affiliations'] = affiliations
    paper_info['authors'] = authors
    seen_papers[paper_info['paper_id']] = None
    out.append(paper_info)

In [4]:
for xml_url in lancet_rss_feeds:
    threads = []
    response = requests.get(xml_url)
    dict_data = xmltodict.parse(response.content)
    try:
        for paper in dict_data['rdf:RDF']['item']:
            thread = threading.Thread(target=get_lancet_info, args=(paper,))
            threads.append(thread)
            thread.start()
        for thread in threads:
            thread.join()
    except:
        print('find me!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print(xml_url)

Exception in thread Thread-173 (get_lancet_info):
Traceback (most recent call last):
  File "c:\Users\ethan\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "c:\Users\ethan\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\ethan\AppData\Local\Temp\ipykernel_93020\2945770657.py", line 16, in get_lancet_info
  File "c:\Users\ethan\AppData\Local\Programs\Python\Python311\Lib\site-packages\selenium\webdriver\remote\webdriver.py", line 455, in close
    self.execute(Command.CLOSE)
  File "c:\Users\ethan\AppData\Local\Programs\Python\Python311\Lib\site-packages\selenium\webdriver\remote\webdriver.py", line 344, in execute
    self.error_handler.check_response(response)
  File "c:\Users\ethan\AppData\Local\Programs\Python\Python311\Lib\site-packages\selenium\webdriver\remote\errorhandler.py", line 229, in check_response
    raise exception_cla

find me!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
https://www.thelancet.com/rssfeed/lanhl_online.xml


Exception in thread Thread-2108 (get_lancet_info):
Traceback (most recent call last):
  File "c:\Users\ethan\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "c:\Users\ethan\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\ethan\AppData\Local\Temp\ipykernel_93020\2945770657.py", line 16, in get_lancet_info
  File "c:\Users\ethan\AppData\Local\Programs\Python\Python311\Lib\site-packages\selenium\webdriver\remote\webdriver.py", line 455, in close
    self.execute(Command.CLOSE)
  File "c:\Users\ethan\AppData\Local\Programs\Python\Python311\Lib\site-packages\selenium\webdriver\remote\webdriver.py", line 344, in execute
    self.error_handler.check_response(response)
  File "c:\Users\ethan\AppData\Local\Programs\Python\Python311\Lib\site-packages\selenium\webdriver\remote\errorhandler.py", line 229, in check_response
    raise exception_cl

In [5]:
# set(paper_info.keys())==set(['paper_id', 'authors', 'subjects', 'title', 'abstract', 'url', 'date', 'affiliations', 'source'])

In [12]:
df = pd.DataFrame.from_records(out)
df['date'] =df['date'].str[:10]
df.to_pickle(r'C:\Users\ethan\Documents\ResearchFinder\daily_temp_files\lancet_temp.pkl')
df

,paper_id,date,source,title,abstract,url,subjects,affiliations,authors
0,10.1016/j.ebiom.2023.104816,2023-10-05,eBioMedicine,Heterogeneous PD-L1 expression in metastases i...,As a groundbreaking approach in cancer treatme...,https://www.thelancet.com/journals/ebiom/artic...,[],"[Faculty of Medicine, School of Biomedical Sci...","[Xiaoqian Hu, Xinpei Deng, Jindong Xie, Hailin..."
1,10.1016/j.ebiom.2023.104819,2023-09-28,eBioMedicine,Pro-tumor Tfh2 cells induce detrimental IgG4 p...,"Our results indicate that in PDAC, highly acti...",https://www.thelancet.com/journals/ebiom/artic...,[Pancreatic cancerTh2 inflammationTfh2IgG4IgEP...,[j These authors contributed equally to this w...,"[Lucia De Monte, Francesca Clemente, Eliana Ru..."
2,10.1016/j.ebiom.2023.104812,2023-10-02,eBioMedicine,Chronic alcohol consumption dysregulates innat...,These data are indicative of aberrant inflamma...,https://www.thelancet.com/journals/ebiom/artic...,[],[],[]
3,10.1016/j.ebiom.2023.104815,2023-10-02,eBioMedicine,Priming with Japanese encephalitis virus or ye...,While a priming vaccination to either JEV or Y...,https://www.thelancet.com/journals/ebiom/artic...,[Japanese encephalitis virus (JEV)Yellow fever...,"[U.S. Military HIV Research Program, Walter Re...","[Yifan Li, Mélanie Merbah, Suzanne Wollen-Robe..."
4,10.1016/j.ebiom.2023.104817,2023-10-05,eBioMedicine,Changes in bone and mineral homeostasis after ...,This exploratory substudy suggests that 12 wee...,https://www.thelancet.com/journals/ebiom/artic...,[Androgen deprivation therapyAndrogen receptor...,"[Clinical and Experimental Endocrinology, Depa...","[Karel David, Gaëtan Devos, Nick Narinx, Leen ..."
...,...,...,...,...,...,...,...,...,...
755,10.1016/j.eclinm.2023.102159,2023-09-14,eClinicalMedicine,Influence of family history on penetrance of h...,Individuals with pathogenic cancer syndrome va...,https://www.thelancet.com/journals/eclinm/arti...,[PenetranceInheritanceFamily historyBRCALynch ...,"[Institute of Biomedical and Clinical Science,...","[Leigh Jackson, Michael N. Weedon, Harry D. Gr..."
756,10.1016/j.eclinm.2023.102230,2023-09-27,eClinicalMedicine,Ranking age-specific modifiable risk factors f...,A large fraction of CVD cases and deaths were ...,https://www.thelancet.com/journals/eclinm/arti...,[Cardiovascular diseaseModifiable factorAgeEpi...,"[Department of Epidemiology, School of Public ...","[Fei Tian, Lan Chen, Zhengmin (Min) Qian, Hui ..."
757,10.1016/j.eclinm.2023.102224,2023-10-02,eClinicalMedicine,Home-based family caregiver-delivered music an...,Our findings suggested that music intervention...,https://www.thelancet.com/journals/eclinm/arti...,[DementiaFamily caregiversMusic interventionsR...,[Creative Arts and Music Therapy Research Unit...,"[Felicity Anne Baker, Vanessa Pac Soo, Jodie B..."
758,10.1016/j.eclinm.2023.102237,2023-10-04,eClinicalMedicine,Efficacy and safety of zapnometinib in hospita...,These results provide proof-of-concept for the...,https://www.thelancet.com/journals/eclinm/arti...,[COVID-19Anti-viralImmunomodulator],"[Goethe University Frankfurt, University Hospi...","[Gernot Rohde, Stephan Stenglein, Hans Prozesk..."


0      2023-10-05
1      2023-09-28
2      2023-10-02
3      2023-10-02
4      2023-10-05
          ...    
755    2023-09-14
756    2023-09-27
757    2023-10-02
758    2023-10-04
759    2023-09-18
Name: date, Length: 760, dtype: object

['10.1016/j.ebiom.2023.104816',
 '10.1016/j.ebiom.2023.104819',
 '10.1016/j.ebiom.2023.104812',
 '10.1016/j.ebiom.2023.104815',
 '10.1016/j.ebiom.2023.104817',
 '10.1016/j.ebiom.2023.104826',
 '10.1016/j.ebiom.2023.104822',
 '10.1016/j.ebiom.2023.104823',
 '10.1016/j.ebiom.2023.104820',
 '10.1016/j.ebiom.2023.104824',
 '10.1016/j.ebiom.2023.104818',
 '10.1016/j.ebiom.2023.104821',
 '10.1016/j.eclinm.2023.102248',
 '10.1016/j.eclinm.2023.102250',
 '10.1016/j.eclinm.2023.102252',
 '10.1016/j.eclinm.2023.102258',
 '10.1016/j.eclinm.2023.102256',
 '10.1016/j.eclinm.2023.102260',
 '10.1016/j.eclinm.2023.102232',
 '10.1016/j.eclinm.2023.102265',
 '10.1016/j.eclinm.2023.102255',
 '10.1016/j.eclinm.2023.102262',
 '10.1016/j.eclinm.2023.102266',
 '10.1016/S0140-6736(23)02221-3',
 '10.1016/S0140-6736(23)01521-0',
 '10.1016/S0140-6736(23)02220-1',
 '10.1016/S0140-6736(23)02080-9',
 '10.1016/S0140-6736(23)01949-9',
 '10.1016/S0140-6736(23)01670-7',
 '10.1016/S0140-6736(23)02193-1',
 '10.1016/S0140